# Estimating Flight Time

In the last section we came up with a scheme to calculate the flight velocity and power required for level flight. In order to estimate the time we might achieve, we need to look at th source of power, our wound-up rubber band. In this section, I will use theory (and assumptions) found in Doug Mclean's 1978 NFFS Symposium article. I will also be using data from Gary Hodson's **Wart-A6** to check to see if we get rational numbers for time.


McLean starts off by assuming that the flight time is proportional to the available energy divided by the thrust power available:

\begin{equation}
t = \eta_{av} \frac{E}{P}
\end{equation}

Where:
    
- **t** is the flight time in seconds
- **E** is the energy stored in the rubber motor
- **P** is the thrust power (thrust x speed) required to stay in level flight
- $\eta_{av}$ is a proportionality constant averaged over the entire flight

Using data from Max Hacklinger, the available energy is a function of the motor weight;

\begin{equation}
E = k_m w_m
\end{equation}Where:

- k_m is a constant = 30,000 inches
- $w_m$ is the motor weight

To get started generating the code for this project, we will set up a few data items from Gary's best flight. We will use the Python **pint** package to track units, so we set that system up first.

In [2]:
import pint
u = pint.UnitRegistry()

In [3]:
wart_best_time = 618 * u.seconds
wart_airframe_weight = 1.2 * u.grams
wart_motor_weight = 0.86 * u.grams
wart_total_weight = wart_airframe_weight + wart_motor_weight
wart_wing_span = (8.0 + 2*3.65)*u.inch
wart_stab_span = 10.8 * u.inch
wart_wing_area = 30.0 * u.inch**2
wart_stab_area = ((1.0 + 1.75)*5.4)*u.inch**2
wart_stab_mean_chord = wart_stab_area / wart_stab_span
wart_wing_mean_chord = wart_wing_area / wart_wing_span
motor_turns = 3660
kiby_dome_elevation = 2160 * u.feet
kiby_dome_height = 147 * u.feet

For the Wart, we get this energy calculation:

In [4]:
k_m = 30000 * u.inches
E = k_m * wart_motor_weight
print(E.to_base_units())

0.65532 kilogram * meter


The thrust power required to maintain level flight balances the model weight against the available lift, and the thrust against the model drag.

\begin{equation}
P = \sqrt{\frac{2w}{\rho S}}\frac{C_l^{3/2}}{C_d}\frac{w_m}{w}
\end{equation}

Where:
- $\rho$ is the air density
- **S** is the total projected area of the model
- $C_l$ is the model lift coefficient
- $C_d$ is the model drag coefficient

For the Wart, we will use these values:

In [5]:
air_density = 33.6 * u.grams/u.foot**3 # from Doug's paper
total_surface_area = wart_wing_area + wart_stab_area
print(total_surface_area)

44.85 inch ** 2


## Calculating Aerodynamic Coefficients

We need to come up with values for your $C_l$ and $C_d$, which are dependent on the model design. Doug made a few simplifying assumptions here.

\begin{equation}
C_{lf} = 1.0
\end{equation}

\begin{equation}
C_{mf} = -0.10
\end{equation}

\begin{equation}
C_{mr} = -0.05
\end{equation}

These values are based on values used in Doug's paper. 

For simplicity, we will assume that the aerodynamic center of the airfoils used on the model are at the quarter chord (average) point on both the wing and stab. These values need to be tuned later.

\begin{equation}
{AR}_{wing} = \frac{b_w^2}{S_f}
\end{equation}

\begin{equation}
{AR}_{stab} = \frac{b_r^2}{S_r}
\end{equation}

Here:
- $b_w$ is the wing span
- $b_r$ is the stab span
- $S_f$ is the wing projected area
- $S_r$ is the stab projected span

In [6]:
aspect_ratio_wing = wart_wing_span**2/wart_wing_area
print(aspect_ratio_wing)

7.803000000000001 dimensionless


In [7]:
aspect_ratio_stab = wart_stab_span**2/wart_stab_area
print(aspect_ratio_stab)

7.8545454545454545 dimensionless


\begin{equation}
C_{lr} = 
    \frac{
        -\frac{x}{{AC}_f}
        -C_{mf}
        -C_{mr}
        \frac{
            S_r {AC}_r
        }
        {
            S_f{AC}_f
        }
   }
   {(1 + \frac{x}{l})(\frac{S_r {AR}_r}{S_f{AR}_f})}
\end{equation}

Where:

x is the distance from the cG to the wing aerodynamic center
l is the distance from the wing and stab aerodynamic centers.
Here are estimates from the Wart plan:

In [8]:
aerodynamic_center_wing = 0.25  * wart_wing_mean_chord
aerodynamic_center_stab = 0.25 * wart_stab_mean_chord
pitching_moment_wing = -0.10
pitching_moment_stab = 0.05
l = ((6 - 1.25)*u.inch - aerodynamic_center_wing) + ((8.5 - 1.75)*u.inch + aerodynamic_center_stab)
x = (1.25 + 2.0) * u.inch
print(aerodynamic_center_wing)
print(aerodynamic_center_stab)
print(x)
print(l)

0.49019607843137253 inch
0.34375 inch
3.25 inch
11.353553921568627 inch


In [49]:
clr1 = - x/aerodynamic_center_wing 
clr2 = pitching_moment_wing 
clr3 = pitching_moment_stab * \
        (wart_stab_area * aerodynamic_center_stab) \
        / (wart_wing_area * aerodynamic_center_wing)
clr4 =  1.0  + x/l
clr5 = (wart_stab_area * l)/(wart_wing_area*aerodynamic_center_wing)
C_lr = (clr1 + clr2 + clr3)/(clr4 * clr5)
print(C_lr)

-0.45519731786882367 dimensionless


Is this correct? We typically adjust the stab to produce a down force, meaning the lift coefficient should be negative to achieve this effect.

The next equation calculates the total model lift coefficient:

\begin{equation}
C_l = \frac{
        1 + \frac{
                S_r C_{lr}
            }{
                S_f
            }
      }
      {
        1+\frac{S_r}{S_f}
      }
\end{equation}

In [50]:
C_l = (1.0 + (wart_stab_area * C_lr)/ wart_wing_area) / \
      (1.0 + (wart_stab_area / wart_wing_area))
print(C_l)

0.5181788144849044 dimensionless


The definition of the lift coefficient is given by:

\begin{equation}
C_l = \frac{2w}{\rho g V^2 S}
\end{equation}

The predicted flight speed calculation is thus given by:
    
\begin{equation}
V = \sqrt{\frac{2 w}{\rho g C_l S}}
\end{equation}

We will need to introduce the acceleration due to gravity to make the units work.

In [11]:
w = wart_airframe_weight + wart_motor_weight
g = 32.2 * u.ft/u.second**2
print(w)
S = wart_wing_area + wart_stab_area
print(S)

2.06 gram
44.85 inch ** 2


In [12]:
V = (2.0 * w * g / (air_density * C_l * S))**0.5
print(V.to_base_units())

1.5075858885046005 meter / second


In [13]:
V.to('miles/hour')

3.372373587385023 <Unit('mile / hour')>

Gary estimated around 2 mph or "walking speed". I walk a bit faster, so this looks reasonable.

The induced drag due to the wing is given by:
\begin{equation}
C_{dif} = \frac{1}{\pi {AR}_f}
\end{equation}

And the induced drag due to the stab is given by:

\begin{equation}
C_{dir} = \frac{C_{lr}^2}{\pi {AR}_r} + 2 C_{lr}C_{dif}
\end{equation}

In [14]:
import math
C_dif = 1.0/(math.pi * aspect_ratio_wing)
C_dir = C_lr ** 2/(math.pi* aspect_ratio_stab) + 2.0*C_lr*C_dif

The total induce drag is given by:

\begin{equation}
C_di = \frac{C_{dif} + \frac{S_r C_{dir}}{S_f}}{1 + \frac{S_r}{S_f}}
\end{equation}

In [15]:
C_di = (C_dif +(wart_stab_area * C_dir)/wart_wing_area)/ \
            (1.0 + wart_stab_area/wart_wing_area)
print(C_di)

0.0177702535214668 dimensionless


### Profile Drag Calculations

McLean uses data from Max Hacklinger to estimate the effects of Reynolds number on the profile drag.

\begin{equation}
C_{dpf} = \frac{6.2}{\sqrt{Re_f}}
\end{equation}

\begin{equation}
C_{dpr} = \frac{4.3 + 1.9 C_{lr}}{\sqrt{Re_r}}
\end{equation}
where:

\begin{equation}
Re_f = \frac{C_f V}{\nu}
\end{equation}

and

\begin{equation}
Re_r = \frac{C_r V}{\nu}
\end{equation}

In [16]:
air_kinematic_viscosity = 15.88e-5 * u.foot ** 2/u.second
Re_f = wart_wing_mean_chord * V / air_kinematic_viscosity
Re_f

5089.38353847144 <Unit('dimensionless')>

In [17]:
Re_r = wart_stab_mean_chord * V / air_kinematic_viscosity
Re_r

3568.930206353097 <Unit('dimensionless')>

In [18]:
C_dpf = 6.2 / math.sqrt(Re_f)
C_dpr = (4.3 + 1.9*C_dpf) / math.sqrt(Re_r)
print(C_dpf)
print(C_dpr)

0.08690786861237122
0.07474197834731462


The total profile drag coefficient is given by:
    
\begin{equation}
C_{dp} = \frac{C_{dpf}+\frac{S_r C_{dpr}}{S_f}}{1 + \frac{S_r}{S_f}}
\end{equation}

In [19]:
C_dp = (C_dpf + wart_stab_area/wart_wing_area * C_dpr)/(1 + wart_stab_area/wart_wing_area)
C_dp

0.08287969758815515 <Unit('dimensionless')>

The drag coefficient of a wing mounting post is approximately:
    
\begin{equation}
C_{dmp} = 1.2
\end{equation}

The final drag coefficient is now given by:
    
\begin{equation}
C_d = C_{di} + C_{dp} + C_{dm}
\end{equation}

In [20]:
wart_wing_post_area = (2.0 * 7.0 /8.0 /16.0) * u.inch**2
C_dmp = 1.2
C_dm = C_dmp * wart_wing_post_area / S
C_d = C_di + C_dp + C_dm
C_d

0.10357637251430422 <Unit('dimensionless')>

## Lift and Thrust Calculation

To maintain level flight, the lift must equal the total model weight and the thrust must equal the total drag. 

In [21]:
lift = air_density * V**2 * S * C_l/(2* g)
lift

2.06 <Unit('gram')>

In [22]:
drag = air_density * V**2 * S * C_d/(2* g)
drag

0.41176389581184336 <Unit('gram')>

The required thrust power is the thrust times the velocity:

In [23]:
P = drag * V
P.to_base_units()

0.0006207694387216136 <Unit('kilogram * meter / second')>

In [48]:
P.to('in oz/sec')

0.8620865181467949 <Unit('inch * ounce / second')>

In [24]:
energy_to_power_ratio = E/P
energy_to_power_ratio.to_base_units()

1055.657638928776 <Unit('second')>

## Calculating Time Factor

The time factor, $\eta_{av}$ is given by:

\begin{equation}
\eta_{av} = F \eta_p
\end{equation}

Here **F** is a fudge factor, and $\eta_p$ is a propeller efficiency factor.

### Von Mises Propeller Efficiency

From the *Theory of Flight* book, Von Mises gives this equation for propeller efficiency:
    
\begin{equation}
\eta_p = \eta_i \frac{1 - \frac{4J\epsilon}{3\pi\eta_i}}{1+\frac{2\pi\eta_i\epsilon}{3J}}    
\end{equation}

Where:
- **J** is the propeller advance ratio $\frac{V}{nd}$
- $\eta_i$ is the propeller induced efficiency
- $\epsilon$ is the propeller profile lift to drag ratio

The propeller induced efficiency is given by:

\begin{equation}
\eta_i = \frac{2 - \frac{J^2\tau}{\pi^2}}{1 + \sqrt{\tau + 1}}
\end{equation}

Where $\tau$ is the thrust loading.

\begin{equation}
\tau = \frac{2 T}{\rho V^2 S_p}
\end{equation}

The area of the prop disk $S_p$ is $36\pi$ for an A-6.

In [38]:
S_p = (9.0 * math.pi)*u.inch**2
S_p

28.274333882308138 <Unit('inch ** 2')>

In [39]:
tau = 2 * drag *g / (air_density * V**2 * S_p)
tau.to_base_units()

0.16429742700935113 <Unit('dimensionless')>

The advance ratio, **J**, is defined as follows:

\begin{equation}
J = \frac{V}{\Omega r}
\end{equation}

Where $\Omega$ is the rotational speed in radians/second, and **r** is the propeller radius.

In [40]:
omega = motor_turns/wart_best_time
omega

5.922330097087379 <Unit('1 / second')>

In [41]:
propeller_radius = 3.0 * u.inch # A-6 rules

In [42]:
J = V / (omega * propeller_radius)
J.to_base_units()

3.3406769613177976 <Unit('dimensionless')>

In [43]:

eta_i = (2 - J**2*tau/math.pi**2)/(math.sqrt(tau + 1.0) + 1.0)
eta_i

0.8726295072645548 <Unit('dimensionless')>

In [44]:
epsilon = 0.1 # assumed from McLean
eta_p = eta_i * (1.0-(4.0*J*epsilon)/(3.0*math.pi*eta_i))/ \
                (1.0+(2.0*eta_i*math.pi*epsilon)/(3.0*J))
eta_p

0.6929372823978628 <Unit('dimensionless')>

All that remains is to come up with the fudge factor **F**. NcLean provides a dimensionless height factor formula and a curve generated from a vatirty of model flight data. 

\begin{equation}
H = \frac{h}{484 \frac{w_m}{w}}
\end{equation}

In [45]:
H = kiby_dome_height/(483 * u.foot * wart_motor_weight/wart_total_weight)
H

0.7290192113245703 <Unit('dimensionless')>

From McLean's curve fit of typical model data, this value would result in an estimated fudge factor of around 0.85.

In [46]:
F = 0.75
eta_av = F * eta_p
eta_av

0.519702961798397 <Unit('dimensionless')>

In [47]:
calculated_time = eta_av * E / P
calculated_time.to_base_units()

548.6284015963877 <Unit('second')>

The actual flight time was 618. This does not match the real flight (a record) but Doug made some assumptions about the lift, and pitching moments for the model, and never considered angle of attack or any other model settings. What I want to do not is combine the work of both of these authors and see what we get. To do that, I need to generalize Erbach's code so we can use **Wart** model data.